In [20]:
import numpy as np
import scipy.integrate as integrate
import sympy as sp

In [2]:
f = lambda x: 4*np.cos(0.5*x)*np.exp(-5/4*x) + 2*np.sin(4.5*x)*np.exp(x/8) + 2

In [3]:
a = 1.3; b = 2.2; alpha = 0; beta = 5/6

In [4]:
p = lambda x, j: (x-a)**(-alpha) * (b-x)**(-beta)*x**j

### Вычисляем моменты весовой функции.

In [6]:
m = []
for j in range(6):
    m.append(integrate.quad(p, a, b, args = (j)))
m = np.array(m)
M = np.array(m[:, 0]); M

array([  5.89555916,  12.2122297 ,  25.56664299,  53.96797923,
       114.65390444, 244.80798315])

Задаем матрицу для решения СЛАУ (п.2)

In [53]:
Mleft = np.array([ [M[0], M[1], M[2]],
                   [M[1], M[2], M[3]],
                   [M[2], M[3], M[4]]
                 ])
Mright = np.array([ [-M[3]], [-M[4]], [-M[5]] 
                  ])
print(Mleft)
print(Mright)

[[  5.89555916  12.2122297   25.56664299]
 [ 12.2122297   25.56664299  53.96797923]
 [ 25.56664299  53.96797923 114.65390444]]
[[ -53.96797923]
 [-114.65390444]
 [-244.80798315]]


### Решаем СЛАУ

In [54]:
A = np.linalg.solve(Mleft, Mright); A

array([[-5.79087259],
       [ 9.82315547],
       [-5.46767826]])

In [55]:
A = A.squeeze(1); A

array([-5.79087259,  9.82315547, -5.46767826])

Находим узлы $X_j,\space j = \overline{1,n} $

In [83]:
x = sp.symbols('x')

In [84]:
omega = x*x*x+A[2]*x*x+A[1]*x+A[0]; omega

x**3 - 5.46767826445881*x**2 + 9.82315546695862*x - 5.79087259260498

In [98]:
x_n = np.array(sp.solve(sp.Eq(omega, 0), x), dtype=complex); x_n

array([1.4312816 +8.47032947e-22j, 1.8533065 +6.61744490e-24j,
       2.18309016-3.30872245e-24j])

In [99]:
x_n

array([1.4312816 +8.47032947e-22j, 1.8533065 +6.61744490e-24j,
       2.18309016-3.30872245e-24j])

Запишем СЛАУ в матричной форме

In [100]:
X = np.array([ [1,    1,    1], 
               [x_n[0]**1, x_n[1]**1, x_n[2]**1], 
               [x_n[0]**2, x_n[1]**2, x_n[2]**2] ]); X

array([[1.        +0.00000000e+00j, 1.        +0.00000000e+00j,
        1.        +0.00000000e+00j],
       [1.4312816 +8.47032947e-22j, 1.8533065 +6.61744490e-24j,
        2.18309016-3.30872245e-24j],
       [2.04856701+2.42468534e-21j, 3.43474499+2.45283073e-23j,
        4.76588267-1.44464789e-23j]])

Решим ее

In [103]:
A = np.linalg.solve(X, M[:3]); A

array([0.39803739+1.26999221e-21j, 1.08877319-1.89525057e-21j,
       4.40874859+6.25258354e-22j])

### Вычисляем интеграл.

In [104]:
integral = np.sum([A[j]*f(x_n[j]) for j in range(3)]); integral

(10.855913096371992-6.818736043361025e-22j)

Смотрим на то, что даёт встроенное интегрирование (ответ тот же, что и через `Wolfram`)

In [105]:
f_p = lambda x: (4*np.cos(0.5*x)*np.exp(-5/4*x) + 2*np.sin(4.5*x)*np.exp(x/8) + 2) * (b-x)**(-beta)

In [106]:
integrate.quad(f_p, a, b)[0]

10.83954513007571

### Абс. погрешность

In [123]:
abs(-integral + integrate.quad(f_p, a, b)[0])

0.016367966296282788

## Вычислим методическую погрешность

### Найдем M3

In [108]:
y = sp.Symbol('y')

In [109]:
f = 4*sp.cos(0.5*y)*sp.exp(-5/4*y) + 2*sp.sin(4.5*y)*sp.exp(y/8) + 2

In [116]:
df = sp.diff(f, y, 6)

In [117]:
df

-16415.4627609253*exp(y/8)*sin(4.5*y) + 2760.80438232422*exp(y/8)*cos(4.5*y) + 18.02734375*exp(-1.25*y)*sin(0.5*y) - 15.5654296875*exp(-1.25*y)*cos(0.5*y)

In [118]:
y_area = np.linspace(a, b, 2000)
val = []
for i in y_area:
    val.append(df.subs(y,i))
M3 = max(abs(max(val)), abs(min(val)))
M3

20807.8098300617

### Вычислим оценку методической погрешности

In [127]:
p_omega = lambda x: abs( (x-x_n[0])*(x-x_n[1])*(x-x_n[2])*(b-x)**(-beta) )

In [131]:
R = M3/np.math.factorial(6) * integrate.quad(p_omega, a, b)[0]

In [132]:
R

1.31022496804320